# Zombie Simulator

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from typing import List, Dict
from matplotlib import animation
from matplotlib import patches
from IPython.display import HTML, Image
import matplotlib
from google.colab import drive
from copy import deepcopy

drive.mount('/content/drive')
matplotlib.rcParams['animation.embed_limit'] = 2**128

## Population Setup

In [ ]:
def random_coords(circle_r, circle_x, circle_y):
  alpha = 2 * math.pi * random.random()
  r = circle_r * math.sqrt(random.random())
  x = abs(math.floor(r * math.cos(alpha) + circle_x))
  y = abs(math.floor(r * math.sin(alpha) + circle_y))
  return x, y

In [ ]:
def random_starts(circle_r: int,circle_x: int,circle_y: int, pop_count: int) -> List:
    starting_points = []
    for i in range(pop_count):
        x, y  = random_coords(circle_r, circle_x, circle_y)
        if [x,y] not in starting_points:
            starting_points.append([x,y])
        else:
            random_coords(circle_r, circle_x, circle_y)
    return starting_points

In [ ]:
def generate_people(start_index: int, coords: List) -> Dict:
    people = {}
    for i in range(len(coords)):
        people[i + start_index] = {}
        people[i + start_index]['coords'] = np.full(
            (1, 2),
            np.array(coords[i])
        )
        people[i + start_index]['state'] = [0]
    return people

In [ ]:
def setup_population(group_count: int, group_size: int, circle_r: int) -> Dict:
    people = {}
    for i in range(1, group_count * 2, 2):
        coords = random_starts(circle_r, group_size * i / 5, group_size * i / 5, group_size)
        group = generate_people(int(group_size * ((i - 1)/2)), coords)
        people.update(group)
    return people

## Generate Walls

In [ ]:
def make_wall(start_coords: List, end_coords: List) -> np.array:
    xs = range(start_coords[0], end_coords[0] + 1) or [start_coords[0]]
    ys = range(start_coords[1], end_coords[1] + 1) or [start_coords[1]]
    return np.array([(x,y) for x in xs for y in ys])

## Check Boundaries

In [ ]:
def check_boundary(coords_to_check: List) -> bool:
    bottom_left = [0,0]
    top_right = [500,500]
    if (coords_to_check[0] >= bottom_left[0] and 
        coords_to_check[0] <= top_right[0] and 
        coords_to_check[1] >= bottom_left[1] and 
        coords_to_check[1] <= top_right[1]):
        return True
    return False

In [ ]:
def check_wall_collision(coords_to_check: List) -> bool:
    if len(walls) == 0:
        return True
    for wall in walls:
        wall_dict = {}
        for coords in wall:
          try:
            wall_dict[coords[0]].append(coords[1])
          except KeyError:
            wall_dict[coords[0]] = [coords[1]]
        try:
          if coords_to_check[0][1] in wall_dict[coords_to_check[0][0]]:
            return False
        except KeyError:
          continue      
    return True

## Movement

In [ ]:
def choose_direction() -> np.array:
    dirs = np.array([[0,1],[0,-1],[1,0],[-1,0]])
    return dirs[random.randrange(4)]

In [ ]:
def take_step(index: int, count: int = 3) -> None:

    status = people[index]['state'][-1]
    if status == 0 and natural_causes():
      set_to_dead(index)
      stats = 4
    if status == 4 and resurrect():
      set_to_zombie(index)
      status = 1

    # This is for normal or immune people
    if status in [0,2]:
      new_coords = [people[index]['coords'][-1] + choose_direction()]
      if check_boundary(new_coords[-1]) and check_wall_collision(new_coords):
        for _ in range(3):
          people[index]['coords'] = np.append(people[index]['coords'], new_coords, axis=0)
          people[index]['state'].append(people[index]['state'][-1])
          check_contact(index)
      else: 
        take_step(index)

    # This is for zombies
    elif status == 1 and count > 0:
      for _ in range(count):
        new_coords = [people[index]['coords'][-1] + choose_direction()]
        if check_boundary(new_coords[-1]) and check_wall_collision(new_coords):
          people[index]['coords'] = np.append(people[index]['coords'], new_coords, axis=0)
          people[index]['state'].append(people[index]['state'][-1])
          check_contact(index)
        else: 
          take_step(index, count - 1)

    # This is for destroyed zombies and people dead of natural causes
    elif status in [3,4]:
      for _ in range(3):
        people[index]['coords'] = np.append(people[index]['coords'], [people[index]['coords'][-1]], axis=0)
        people[index]['state'].append(people[index]['state'][-1])

## Interactions

In [ ]:
def check_contact(index: int) -> None:
    for key in people.keys():
        if key <= index:
            continue
        if np.array_equal(people[index]['coords'][-1], people[key]['coords'][-1]):
            check_states(index, key)

In [ ]:
def check_states(index: int, key: int) -> None:
    person_a = people[index]
    person_b = people[key]
    if person_a['state'][-1] == 1 and person_b['state'][-1] == 0:
        bite(key, index)
    elif person_a['state'][-1] == 0 and person_b['state'][-1] == 1:
        bite(index, key)

In [ ]:
def bite(human: int, zombie: int) -> None:
    if zombie_killed():
      print(f"Zombie {zombie} killed")
      people[zombie]['state'][-1] = 3
      return
    if transmission():
      print(f"Human {human} bitten")
      people[human]['state'][-1] = 1
    else:
      print(f"Human {human} is immune!")
      people[human]['state'][-1] = 2

In [ ]:
def transmission() -> int:
  return random.randint(1,500) > 5

In [ ]:
def zombie_killed() -> int:
  return random.randint(1,50) > 40

In [ ]:
def natural_causes() -> int:
  return random.randint(1,50000) > 49999

In [ ]:
def set_to_dead(index: int) -> None:
  print(f"Human {index} died of natural causes")
  people[index]['state'][-1] = 4

In [ ]:
def resurrect() -> int:
  return random.randint(1,1000) > 999

In [ ]:
def set_to_zombie(index) -> None:
  print(f"Human {index} has risen from the dead")
  people[index]['state'][-1] = 1

## Utils

In [ ]:
def view_starting_configuration() -> None:
  fig, ax = plt.subplots(figsize=(8,8))
  ax.set_xlim(0,500)
  ax.set_ylim(0,500)
  for index in people.keys():
    if people[index]['state'][0] == 0:
      sym = 'g+'
    elif people[index]['state'][0] == 1:
      sym = 'r+'
    ax.plot(people[index]['coords'][0,0], people[index]['coords'][0,1], sym)
  for wall_coords in walls:
    x = [coords[0] for coords in wall_coords]
    y = [coords[1] for coords in wall_coords]
    ax.plot(x, y, 'b')
  human = patches.Patch(color='g', label='Human')
  zombie = patches.Patch(color='r', label='Zombie')
  immune = patches.Patch(color='k', label='Immune Human')
  destroyed = patches.Patch(color='c', label='Destroyed Zombie')
  dead = patches.Patch(color='m', label='Dead Human (Natural Causes)')
  ax.legend(handles=[human,zombie,immune,destroyed,dead],loc='upper left', fontsize='x-small')

In [ ]:
def save_progress(number: int) -> None:
  with open(f'/content/drive/My Drive/zombies/zombies-{number}.npy', 'wb') as f:
    np.save(f, people)
  print("SAVED!")

In [ ]:
def load_progress(number: int) -> Dict:
  with open(f'/content/drive/My Drive/zombies/zombies-{number}.npy', 'rb') as f:
    people = np.load(f, allow_pickle=True)
  print("LOADED!")
  return dict(people.tolist())

In [ ]:
def slice_people(start: int, end: int) -> Dict:
  people = {}
  for index in people_cp.keys():
    people[index] = {}
    people[index]['coords'] = np.array(people_cp[index]['coords'][start:end])
    people[index]['state'] = np.array(people_cp[index]['state'][start:end])
  return people

In [ ]:
def count_infected() -> int:
    bitten = []
    for i in people.keys():
        if people[i]['state'][-1] == 1:
            bitten.append(1)
    return len(bitten)

## Setting Up Simulation

In [ ]:
wall_one = make_wall([400,0],[400,300])
wall_two = make_wall([0,300],[200,300])
wall_three = make_wall([325,300],[400,300])
walls = [wall_one, wall_two, wall_three]
people = setup_population(2, 500, 100)
people[0]['state'][0] = 1
people[1]['state'][0] = 1
people[2]['state'][0] = 1
people[3]['state'][0] = 1
people[4]['state'][0] = 1
people[5]['state'][0] = 1
people[6]['state'][0] = 1
people[7]['state'][0] = 1
view_starting_configuration()

## Running Simulation

To save on resources, each chunk is run for 500 steps which generates 1500 entries into the peoples' records

In [ ]:
for i in range(500):
    if i % 20 == 0:
        print(f"COMPLETED {(i/500)*100:.2f}%")
        print(f"Total Infected: {count_infected()}")
    for index in people.keys():
        take_step(index)
save_progress(5)
print("COMPLETED 100.00%")

## Preparing A Slice Of Data For Animation

In [ ]:
people_cp = deepcopy(people)

In [ ]:
for index in people_cp.keys():
  people_cp[index]['state'] = np.array(people_cp[index]['state'])

In [ ]:
sliced_people = slice_people(0,1500)

## Generating Animation

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
ax.set_xlim(0,500)
ax.set_ylim(0,500)

In [ ]:
def run(i):
  ax.cla()
  ax.set_xlim(0,500)
  ax.set_ylim(0,500)
  for index in sliced_people.keys():
    if sliced_people[index]['state'][i] == 0:
      sym = 'g+'
    elif sliced_people[index]['state'][i] == 1:
      sym = 'r+'
    elif sliced_people[index]['state'][i] == 2:
      sym = 'k+'
    elif sliced_people[index]['state'][i] == 3:
      sym = 'c+'
    elif sliced_people[index]['state'][i] == 4:
      sym = 'm+'
    ax.plot(sliced_people[index]['coords'][i,0], sliced_people[index]['coords'][i,1], sym)
  for wall_coords in walls:
    x = [coords[0] for coords in wall_coords]
    y = [coords[1] for coords in wall_coords]
    ax.plot(x, y, 'b')
  human = patches.Patch(color='g', label='Human')
  zombie = patches.Patch(color='r', label='Zombie')
  immune = patches.Patch(color='k', label='Immune Human')
  destroyed = patches.Patch(color='c', label='Destroyed Zombie')
  dead = patches.Patch(color='m', label='Dead Human (Natural Causes)')
  ax.legend(handles=[human,zombie,immune,destroyed,dead],loc='upper left', fontsize='x-small')

In [ ]:
anim = animation.FuncAnimation(fig, run, frames=150, interval=50)

In [ ]:
HTML(anim.to_html5_video())